# Gesture Recognition
In this group project, you are going to build a 2D CNN + RNN model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
#we will use cv2 here
import cv2

import datetime
import os
import matplotlib.pyplot as plt
from keras.applications import ResNet50

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('train.csv').readlines())
val_doc = np.random.permutation(open('val.csv').readlines())
print(len(train_doc), len(val_doc))
batch_size = 50 #experiment with the batch size

663 100


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
scale_f = 70

### We have modified the generator function by adding 2 more parameters scale_f -> size of image to be rescaled and n_i -> number of images to train on

In [8]:
def generator(source_path, folder_list, batch_size, scale_f, n_i):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #create a list of image numbers you want to use for a particular video
    img_idx = [i for i in range(30)]
    if(n_i == 20):
        img_idx = [0,1,4,5,7,8,10,11,13,14,16,17,19,20,22,23,25,26,28,29]
    elif(n_i == 15):
        img_idx = [i for i in range(0,30,2)]
    while True:
        t = np.random.permutation(folder_list)
        # calculate the number of batches
        num_batches = int(len(folder_list)/batch_size)
    
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),scale_f,scale_f,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    #We have 160*120 and 360*360 images, we will resize every image to scale_f*scale_f
                    image = cv2.resize(image, (scale_f, scale_f))
                    
                    
                    #normalising using min-max method
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - np.min(image[:,:,0])/np.max(image[:,:,0]) - np.min(image[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - np.min(image[:,:,1])/np.max(image[:,:,1]) - np.min(image[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - np.min(image[:,:,2])/np.max(image[:,:,2]) - np.min(image[:,:,2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        left_samples = len(folder_list)%batch_size
        offset = len(folder_list) - left_samples
        if (left_samples > 0):
            batch_data = np.zeros((left_samples,len(img_idx),scale_f,scale_f,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((left_samples,5)) # batch_labels is the one hot representation of the output
            for folder in range(left_samples): # iterate over the left_samples
                imgs = os.listdir(source_path+'/'+ t[folder + offset].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + offset].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    #We have 160*120 and 360*360 images, we will resize every image to scale_f*scale_f
                    image = cv2.resize(image, (scale_f, scale_f))


                    #normalising using min-max method
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - np.min(image[:,:,0])/np.max(image[:,:,0]) - np.min(image[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - np.min(image[:,:,1])/np.max(image[:,:,1]) - np.min(image[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - np.min(image[:,:,2])/np.max(image[:,:,2]) - np.min(image[:,:,2])#normalise and feed in the image

                batch_labels[folder, int(t[folder + offset].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [9]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 1# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 1


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [10]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, LSTM, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2

In [26]:
#write your model here
#initially building a simple model to test whether model is fitting and generator is running or not
model = Sequential()
n_i = 30
model.add(TimeDistributed(Conv2D(64,(3,3),activation='relu',input_shape=(n_i, scale_f, scale_f, 3))))#(inputs))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

# flatten
model.add(TimeDistributed(Flatten()))

#addig GRU layers
model.add(GRU(64))
model.add(Dropout(0.25))
model.add(Dense(100, activation='relu'))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [9]:
optimiser = 'sgd'#write your optimizer

#building model with input
input_param = (None, n_i, scale_f,scale_f,3)
model.build(input_param)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 68, 68, 64)    1792      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 34, 34, 64)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 34, 34, 64)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 73984)         0         
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                14217408  
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               6500      
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [11]:
train_generator = generator(train_path, train_doc, batch_size, scale_f, n_i)
val_generator = generator(val_path, val_doc, batch_size, scale_f, n_i)

In [12]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001) # write the Reducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [14]:
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 50
Source path =  train ; batch size = 50
Epoch 1/1
14/14 [==============================] - 207s 15s/step - loss: 1.7145 - categorical_accuracy: 0.1964 - val_loss: 1.6298 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2020-05-24--09_28_58.161775/model-00001-1.71857-0.19457-1.62976-0.20000.h5


In [15]:
#since Last model is very bad in terms of accuracy, we will add more conv2d timedistributed layers and increase the epochs

#write your model here
model = Sequential()
n_i = 30
model.add(TimeDistributed(Conv2D(64,(3,3),activation='relu',input_shape=(n_i, scale_f, scale_f, 3))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(32, (3, 3))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same')))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(64, (3, 3))))
model.add(TimeDistributed((Activation('relu'))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.25)))


# flatten
model.add(TimeDistributed(Flatten()))

#adding GRU layer with dropout
model.add(GRU(64,return_sequences=True))
model.add(TimeDistributed(Dropout(0.25)))

#flattening again for dense input
model.add(Flatten())

#taking a single dense before output softmax
model.add(Dense(100, activation='relu'))
model.add(Dense(5, activation='softmax'))

optimiser = 'sgd'#write your optimizer

input_param = (None, n_i, scale_f,scale_f,3)
model.build(input_param)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_5 (TimeDist (None, 30, 68, 68, 64)    1792      
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 68, 68, 64)    0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 68, 68, 64)    256       
_________________________________________________________________
time_distributed_8 (TimeDist (None, 30, 66, 66, 32)    18464     
_________________________________________________________________
time_distributed_9 (TimeDist (None, 30, 66, 66, 32)    0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, 30, 66, 66, 32)    128       
_________________________________________________________________
time_distributed_11 (TimeDis (None, 30, 33, 33, 32)    0         
__________

In [15]:
#changing batch size to fit in memory
batch_size = 30

#do for 10 epochs
num_epochs = 10
train_generator = generator(train_path, train_doc, batch_size,scale_f,n_i)
val_generator = generator(val_path, val_doc, batch_size,scale_f,n_i)

#save model weights
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001) # write the Reducelronplateau code here
callbacks_list = [checkpoint, LR]


#steps per epoch
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
#fit model
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 30
Source path =  train ; batch size = 30
Epoch 1/10
23/23 [==============================] - 56s 2s/step - loss: 1.4835 - categorical_accuracy: 0.3729 - val_loss: 1.5458 - val_categorical_accuracy: 0.4600

Epoch 00001: saving model to model_init_2020-05-24--08_48_12.665996/model-00001-1.53080-0.34842-1.54583-0.46000.h5
Epoch 2/10
23/23 [==============================] - 46s 2s/step - loss: 1.1930 - categorical_accuracy: 0.5026 - val_loss: 1.2901 - val_categorical_accuracy: 0.5100

Epoch 00002: saving model to model_init_2020-05-24--08_48_12.665996/model-00002-1.21242-0.49623-1.29014-0.51000.h5
Epoch 3/10
23/23 [==============================] - 48s 2s/step - loss: 0.9713 - categorical_accuracy: 0.6100 - val_loss: 1.1817 - val_categorical_accuracy: 0.5300

Epoch 00003: saving model to model_init_2020-05-24--08_48_12.665996/model-00003-0.97719-0.60784-1.18171-0.53000.h5
Epoch 4/10
23/23 [==============================] - 46s 2s/step - loss: 0.9142 - cat

In [20]:
#since Last model gave 54% validation accuracy at max
#let's check with another model with 30 images and 70 as size of scaled images
scale_f = 70
#write your model here
model = Sequential()
n_i = 30
model.add(TimeDistributed(Conv2D(64,(3,3),activation='relu',input_shape=(n_i, scale_f, scale_f, 3))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(32, (3, 3))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same')))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(64, (3, 3))))
model.add(TimeDistributed((Activation('relu'))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.25)))


#add another set of conv2d timedistributed layers for better feature extraction
model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same')))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(128, (3, 3))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

# flatten
model.add(TimeDistributed(Flatten()))

#adding GRU layers
model.add(GRU(64,return_sequences=True))


model.add(Flatten())

#increasing the dense layer size to 512 for better learning
model.add(Dense(512,kernel_regularizer=l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

optimiser = 'sgd'#write your optimizer

input_param = (None, n_i, scale_f,scale_f,3)
model.build(input_param)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_75 (TimeDis (None, 30, 68, 68, 64)    1792      
_________________________________________________________________
time_distributed_76 (TimeDis (None, 30, 68, 68, 64)    0         
_________________________________________________________________
time_distributed_77 (TimeDis (None, 30, 68, 68, 64)    256       
_________________________________________________________________
time_distributed_78 (TimeDis (None, 30, 66, 66, 32)    18464     
_________________________________________________________________
time_distributed_79 (TimeDis (None, 30, 66, 66, 32)    0         
_________________________________________________________________
time_distributed_80 (TimeDis (None, 30, 66, 66, 32)    128       
_________________________________________________________________
time_distributed_81 (TimeDis (None, 30, 33, 33, 32)    0         
__________

In [20]:
batch_size = 20
num_epochs = 10
train_generator = generator(train_path, train_doc, batch_size,scale_f, n_i)
val_generator = generator(val_path, val_doc, batch_size,scale_f, n_i)

#save model weights
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001) # write the Reducelronplateau code here
callbacks_list = [checkpoint, LR]


#steps per epoch
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
#fit model
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  train ; batch size = 20
 Epoch 1/10val
 ; batch size = 20
34/34 [==============================] - 47s 1s/step - loss: 9.8421 - categorical_accuracy: 0.3282 - val_loss: 9.5226 - val_categorical_accuracy: 0.3600

Epoch 00001: saving model to model_init_2020-05-24--08_48_12.665996/model-00001-9.86147-0.31976-9.52258-0.36000.h5
Epoch 2/10
34/34 [==============================] - 41s 1s/step - loss: 9.4809 - categorical_accuracy: 0.3885 - val_loss: 9.2407 - val_categorical_accuracy: 0.4000

Epoch 00002: saving model to model_init_2020-05-24--08_48_12.665996/model-00002-9.43744-0.39819-9.24071-0.40000.h5
Epoch 3/10
34/34 [==============================] - 42s 1s/step - loss: 9.1201 - categorical_accuracy: 0.4915 - val_loss: 8.8602 - val_categorical_accuracy: 0.5400

Epoch 00003: saving model to model_init_2020-05-24--08_48_12.665996/model-00003-9.13411-0.48718-8.86024-0.54000.h5
Epoch 4/10
34/34 [==============================] - 43s 1s/step - loss: 8.8320 - cat

In [ ]:
#now let's again check with decreasing learning rate at each epoch (patience = 1)
batch_size = 20
num_epochs = 20
train_generator = generator(train_path, train_doc, batch_size,scale_f, n_i)
val_generator = generator(val_path, val_doc, batch_size,scale_f, n_i)

#save model weights
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001) # write the Reducelronplateau code here
callbacks_list = [checkpoint, LR]


#steps per epoch
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
#fit model
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 20
Source path =  train ; batch size = 20
Epoch 1/20
34/34 [==============================] - 47s 1s/step - loss: 9.6873 - categorical_accuracy: 0.3584 - val_loss: 9.8646 - val_categorical_accuracy: 0.2400

Epoch 00001: saving model to model_init_2020-05-24--09_28_58.161775/model-00001-9.70375-0.35897-9.86457-0.24000.h5
Epoch 2/20
34/34 [==============================] - 39s 1s/step - loss: 9.3652 - categorical_accuracy: 0.4224 - val_loss: 9.3455 - val_categorical_accuracy: 0.5400

Epoch 00002: saving model to model_init_2020-05-24--09_28_58.161775/model-00002-9.35713-0.43288-9.34547-0.54000.h5
Epoch 3/20
34/34 [==============================] - 42s 1s/step - loss: 8.9759 - categorical_accuracy: 0.5386 - val_loss: 9.6180 - val_categorical_accuracy: 0.4000

Epoch 00003: saving model to model_init_2020-05-24--09_28_58.161775/model-00003-8.98995-0.53544-9.61797-0.40000.h5
Epoch 4/20
34/34 [==============================] - 42s 1s/step - loss: 8.7133 - cat

In [21]:
#as even test accuracy is not getting better well, seems some problem with feature extraction part

#let's try transfer learning with Resnet50

scale_f = 198 #size of image limitation with Resnet is 197
from keras.applications import ResNet50

#write your model here
# create a Sequential model
model = Sequential()
n_i = 15  # model was getting OOM

#create base resnet model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(scale_f, scale_f, 3))

#making some layers trainable
split_at = 160
for layer in base_model.layers[:split_at]: layer.trainable = False
for layer in base_model.layers[split_at:]: layer.trainable = True

model.add(TimeDistributed(base_model, input_shape=(n_i, scale_f, scale_f, 3)))
model.add(TimeDistributed(GlobalAveragePooling2D()))
# flatten
model.add(TimeDistributed(Flatten()))

#adding GRU layers
model.add(GRU(100,return_sequences=True))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(5, activation='softmax'))

optimiser = 'sgd' #write your optimizer

input_param = (None, n_i, scale_f,scale_f,3)
model.build(input_param)
print(model.summary())

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_6 (TimeDist (None, 15, 7, 7, 2048)    23587712  
_________________________________________________________________
time_distributed_7 (TimeDist (None, 15, 2048)          0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 15, 2048)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 15, 100)           644700    
_________________________________________________________________
flatten_4 (Flatten)          (None, 1500)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               150100    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
__________

In [22]:
#as size of image is increased, we have to decrease batch size to 20 to get the batch tensor fit in nimblebox GPU machine
batch_size = 20
num_epochs = 10
train_generator = generator(train_path, train_doc, batch_size,scale_f, n_i)
val_generator = generator(val_path, val_doc, batch_size,scale_f, n_i)

#save model weights
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001) # write the Reducelronplateau code here
callbacks_list = [checkpoint, LR]


#steps per epoch
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
#fit model
# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate when we are fine tuning the initial layers
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['categorical_accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 20
Source path =  trainEpoch 1/10
 ; batch size = 20
34/34 [==============================] - 173s 5s/step - loss: 1.5034 - categorical_accuracy: 0.3002 - val_loss: 1.3627 - val_categorical_accuracy: 0.3700

Epoch 00001: saving model to model_init_2020-05-24--10_26_27.412450/model-00001-1.50548-0.30769-1.36273-0.37000.h5
Epoch 2/10
34/34 [==============================] - 91s 3s/step - loss: 0.8330 - categorical_accuracy: 0.7307 - val_loss: 1.1091 - val_categorical_accuracy: 0.5300

Epoch 00002: saving model to model_init_2020-05-24--10_26_27.412450/model-00002-0.83283-0.72398-1.10905-0.53000.h5
Epoch 3/10
34/34 [==============================] - 91s 3s/step - loss: 0.4725 - categorical_accuracy: 0.8595 - val_loss: 1.2309 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2020-05-24--10_26_27.412450/model-00003-0.46144-0.86425-1.23095-0.50000.h5
Epoch 4/10
34/34 [==============================] - 91s 3s/step - loss: 0.2899 - ca

In [35]:
# We got 75% validation accuracy for 7th epoch which is pretty good, 
# it seems that fature extraction using resnet is good, let's make the model learn more
# now we should fine tune this model giving 20 images and train over 150th layer
# also making GRU layer a little bigger and adding another timedistributed dense layer after it for better learning

#let's try transfer learning with Resnet
scale_f = 197  #size of image limitation with Resnet is 197
n_i = 20
# write your model here
# create a Sequential model
model = Sequential()


base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(scale_f, scale_f, 3))

#making some layers trainable
split_at = 150
for layer in base_model.layers[:split_at]: layer.trainable = False
for layer in base_model.layers[split_at:]: layer.trainable = True

model.add(TimeDistributed(base_model, input_shape=(n_i, scale_f, scale_f, 3)))
model.add(TimeDistributed(GlobalAveragePooling2D()))
# flatten
model.add(TimeDistributed(Flatten()))

#adding GRU/LSTM layers
model.add(GRU(1000,return_sequences=True))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dropout(0.30)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(5, activation='softmax'))

optimiser = 'sgd' #write your optimizer

input_param = (None, n_i, scale_f,scale_f,3)
model.build(input_param)
print(model.summary())

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_38 (TimeDis (None, 20, 7, 7, 2048)    23587712  
_________________________________________________________________
time_distributed_39 (TimeDis (None, 20, 2048)          0         
_________________________________________________________________
time_distributed_40 (TimeDis (None, 20, 2048)          0         
_________________________________________________________________
gru_8 (GRU)                  (None, 20, 1000)          9147000   
_________________________________________________________________
time_distributed_41 (TimeDis (None, 20, 100)           100100    
_________________________________________________________________
time_distributed_42 (TimeDis (None, 20, 100)           0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 2000)              0         
__________

In [36]:
#increasing epochs and decreasing learning rate to 0.0004

batch_size = 20
num_epochs = 15
train_generator = generator(train_path, train_doc, batch_size,scale_f, n_i)
val_generator = generator(val_path, val_doc, batch_size,scale_f, n_i)

#save model weights
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.0004) # write the Reducelronplateau code here
callbacks_list = [checkpoint, LR]


#steps per epoch
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
#fit model
# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate when we are fine tuning the initial layers
sgd = optimizers.SGD(lr=0.0004, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['categorical_accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 20
Source path =  Epoch 1/15train ; batch size = 20

34/34 [==============================] - 174s 5s/step - loss: 1.6110 - categorical_accuracy: 0.2664 - val_loss: 1.5214 - val_categorical_accuracy: 0.2600

Epoch 00001: saving model to model_init_2020-05-24--10_26_27.412450/model-00001-1.61835-0.25641-1.52141-0.26000.h5
Epoch 2/15
34/34 [==============================] - 150s 4s/step - loss: 1.3561 - categorical_accuracy: 0.4452 - val_loss: 1.3496 - val_categorical_accuracy: 0.3400

Epoch 00002: saving model to model_init_2020-05-24--10_26_27.412450/model-00002-1.35980-0.44796-1.34958-0.34000.h5
Epoch 3/15
34/34 [==============================] - 130s 4s/step - loss: 1.0241 - categorical_accuracy: 0.6255 - val_loss: 1.1863 - val_categorical_accuracy: 0.4300

Epoch 00003: saving model to model_init_2020-05-24--10_26_27.412450/model-00003-1.02665-0.62443-1.18629-0.43000.h5
Epoch 4/15
34/34 [==============================] - 130s 4s/step - loss: 0.8204 -

## Final Model

In [14]:
# We got 80% validation accuracy for 10th epoch which is very nice
# now we should fine tune this model more,  train over 140th layer
# also changing GRU layer to LSTM

scale_f = 197 #size of image limitation with Resnet
n_i = 20
# write your model here
# create a Sequential model
model = Sequential()

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(scale_f, scale_f, 3))

#making some layers trainable
split_at = 140
for layer in base_model.layers[:split_at]: layer.trainable = False
for layer in base_model.layers[split_at:]: layer.trainable = True

model.add(TimeDistributed(base_model, input_shape=(n_i, scale_f, scale_f, 3)))
model.add(TimeDistributed(GlobalAveragePooling2D()))
# flatten
model.add(TimeDistributed(Flatten()))

#adding GRU/LSTM layers of more units and adding timedistributed dense layer of 128 neurons each
model.add(LSTM(1024,return_sequences=True))
model.add(TimeDistributed(Dense(128, activation='relu')))
model.add(TimeDistributed(Dropout(0.30)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(5, activation='softmax'))

optimiser = 'sgd' #write your optimizer

input_param = (None, n_i, scale_f,scale_f,3)
model.build(input_param)
print(model.summary())

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_7 (TimeDist (None, 20, 7, 7, 2048)    23587712  
_________________________________________________________________
time_distributed_8 (TimeDist (None, 20, 2048)          0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 20, 2048)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 1024)          12587008  
_________________________________________________________________
time_distributed_10 (TimeDis (None, 20, 128)           131200    
_________________________________________________________________
time_distributed_11 (TimeDis (None, 20, 128)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 2560)              0         
__________

In [16]:
#increasing epochs to 25 and making learning rate to 0.0005

batch_size = 20
num_epochs = 25
train_generator = generator(train_path, train_doc, batch_size,scale_f, n_i)
val_generator = generator(val_path, val_doc, batch_size,scale_f, n_i)

#save model weights
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.0005) # write the Reducelronplateau code here
callbacks_list = [checkpoint, LR]


#steps per epoch
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
#fit model
# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate when we are fine tuning the initial layers
sgd = optimizers.SGD(lr=0.0005, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['categorical_accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size =Source path =  train ; batch size = 20
 Epoch 1/25
20
34/34 [==============================] - 204s 6s/step - loss: 1.5403 - categorical_accuracy: 0.2973 - val_loss: 1.5294 - val_categorical_accuracy: 0.3200

Epoch 00001: saving model to model_init_2020-05-24--12_25_32.551532/model-00001-1.54777-0.28808-1.52944-0.32000.h5
Epoch 2/25
34/34 [==============================] - 132s 4s/step - loss: 1.2556 - categorical_accuracy: 0.5320 - val_loss: 1.3551 - val_categorical_accuracy: 0.4800

Epoch 00002: saving model to model_init_2020-05-24--12_25_32.551532/model-00002-1.25346-0.53695-1.35507-0.48000.h5
Epoch 3/25
34/34 [==============================] - 132s 4s/step - loss: 0.8082 - categorical_accuracy: 0.7616 - val_loss: 0.9590 - val_categorical_accuracy: 0.5200

Epoch 00003: saving model to model_init_2020-05-24--12_25_32.551532/model-00003-0.81318-0.75566-0.95900-0.52000.h5
Epoch 4/25
34/34 [==============================] - 132s 4s/step - loss: 0.5026 -

RuntimeError: Problems closing file (file write failed: time = Sun May 24 13:28:19 2020
, filename = 'model_init_2020-05-24--12_25_32.551532/model-00020-0.02290-0.99397-0.40433-0.82000.h5', file descriptor = 63, errno = 28, error message = 'No space left on device', buf = 0x559e0bbd37b0, total write size = 2048, bytes this sub-write = 2048, bytes actually written = 18446744073709551615, offset = 4096)

### above program closed due to lack of memory in nimblebox but we were getting close to high accuracy

### But completing previous training for next few iterations more

In [17]:
#increasing epochs to 25 and making learning rate to 0.0005

batch_size = 20
num_epochs = 25
train_generator = generator(train_path, train_doc, batch_size,scale_f, n_i)
val_generator = generator(val_path, val_doc, batch_size,scale_f, n_i)

#save model weights
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','--').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.0005) # write the Reducelronplateau code here
callbacks_list = [checkpoint, LR]


#steps per epoch
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
#fit model
# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate when we are fine tuning the initial layers
sgd = optimizers.SGD(lr=0.0005, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['categorical_accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 20
Source path =  train ; batch size = 20
Epoch 1/25
34/34 [==============================] - 146s 4s/step - loss: 0.0120 - categorical_accuracy: 0.9971 - val_loss: 0.2736 - val_categorical_accuracy: 0.8500

Epoch 00001: saving model to model_init_2020-05-24--12_25_32.551532/model-00001-0.01175-0.99698-0.27365-0.85000.h5
Epoch 2/25
34/34 [==============================] - 132s 4s/step - loss: 0.0226 - categorical_accuracy: 0.9890 - val_loss: 0.3383 - val_categorical_accuracy: 0.8700

Epoch 00002: saving model to model_init_2020-05-24--12_25_32.551532/model-00002-0.01090-0.99698-0.33826-0.87000.h5
Epoch 3/25
34/34 [==============================] - 132s 4s/step - loss: 0.0146 - categorical_accuracy: 0.9971 - val_loss: 0.4367 - val_categorical_accuracy: 0.8200

Epoch 00003: saving model to model_init_2020-05-24--12_25_32.551532/model-00003-0.01386-0.99698-0.43671-0.82000.h5
Epoch 4/25
34/34 [==============================] - 132s 4s/step - loss: 0.0146 -

### Now we have obtained an accuracy over validation set of 91%
### We will end furthur research for now as this seems a pretty good accuracy